In [4]:
#!uv add openai-agents

In [1]:
import agents

In [2]:
agents.__version__

'0.4.0'

In [3]:
#!uv remove openai

In [4]:
#!uv add openai-agents

In [5]:
from agents import Agent, function_tool, Runner

In [6]:
import requests

In [7]:

url = 'https://www.stromnetz.berlin'
def fetch_url(url):
    jina_reader_url = base_url + url
    response = requests.get(jina_reader_url)
    return response.content.decode('utf-8')

In [8]:
fetch_url(url)

NameError: name 'base_url' is not defined

In [9]:
import requests
from requests.exceptions import RequestException, Timeout, HTTPError

def fetch_url(url: str) -> str:
    """
    Fetches the content of a given URL by appending it to the base URL and performing an HTTP GET request.

    Args:
        url (str): The relative or absolute URL path to fetch.
        

    Returns:
        str: The decoded text content of the response.

    Raises:
        ValueError: If the URL or response content is invalid or cannot be decoded.
        HTTPError: If the HTTP request returns an unsuccessful status code.
        Timeout: If the request times out.
        RequestException: For general network-related errors.
    """
    base_url = 'https://r.jina.ai/'
    jina_reader_url = base_url + url
    try:
        response = requests.get(jina_reader_url, timeout=10)
        response.raise_for_status()  # Raises HTTPError for bad status codes
        try:
            return response.content.decode("utf-8")
        except UnicodeDecodeError as e:
            raise ValueError(f"Failed to decode response content from {jina_reader_url}") from e
    except Timeout as e:
        raise Timeout(f"Request to {jina_reader_url} timed out.") from e
    except HTTPError as e:
        raise HTTPError(f"HTTP error occurred while fetching {jina_reader_url}: {e}") from e
    except RequestException as e:
        raise RequestException(f"Error occurred while requesting {jina_reader_url}: {e}") from e


In [10]:
fetch_url(url)

'Title: Stromnetz Berlin | Berlins zuverlässiger Netzbetreiber\n\nURL Source: https://www.stromnetz.berlin/\n\nMarkdown Content:\nStromnetz Berlin | Berlins zuverlässiger Netzbetreiber\n\n===============\n[Zum Hauptinhalt springen](https://www.stromnetz.berlin/#_main)[Zur Navigation springen](https://www.stromnetz.berlin/#_mainnav)[Zur Suche springen](https://www.stromnetz.berlin/#)\n\nGeöffnen Shclisen\n\n[![Image 1: Logotype](https://www.stromnetz.berlin/files/ui/images/logo/logotype.svg)](https://www.stromnetz.berlin/)\n\n*   Anschließen \n    *   [Anschließen](https://www.stromnetz.berlin/anschliessen/)\n    *   [Anschluss Niederspannung](https://www.stromnetz.berlin/anschliessen/anschluss-niederspannung/)\n        *   [Inspektion Netzanschluss](https://www.stromnetz.berlin/anschliessen/anschluss-niederspannung/inspektion-netzanschluss/)\n\n    *   [Anschluss Mittel- und Hochspannung](https://www.stromnetz.berlin/anschliessen/anschluss-mittel-hochspannung/)\n        *   [Anlagensic

In [11]:
web_agent= Agent(
    name='web_agent',
    instructions='you are a helpful assistant',
    model='gpt-4o-mini',
    tools = [function_tool(fetch_url)]
)
    

In [12]:
question="What is this page about? https://www.stromnetz.berlin"

In [13]:
runner = Runner()


In [14]:
results = await runner.run(web_agent,input=question)

In [16]:
results.final_output

"The webpage for **Stromnetz Berlin** serves as the official site for Berlin's electricity grid operator. Here are key aspects covered:\n\n1. **Connection Services**:\n   - Information on how to connect to the electricity network, including for low, medium, and high voltage.\n   - Details regarding electric mobility and relevant infrastructure.\n\n2. **Feeding Electricity**:\n   - Guidelines for feeding renewable energy back into the grid, particularly solar energy.\n   - Management practices for energy input.\n\n3. **Meter Services**:\n   - Information about electricity meters, including installation, monitoring, and customer service related to metering.\n\n4. **Network Utilization**:\n   - Details on how customers can utilize the electricity network, including tariffs and other services.\n\n5. **Public Lighting**:\n   - Information about street lighting management and projects.\n\n6. **Career Opportunities**:\n   - Job listings and information about training opportunities within the 

In [5]:
import requests

def get_page_content(url: str) -> str:
    """
    Fetch the textual content of a webpage using the Jina AI Reader service.

    This function prefixes the provided URL with Jina AI's reader endpoint 
    (`https://r.jina.ai/`), sends a GET request to that endpoint, and returns 
    the UTF-8 decoded content of the response.

    Args:
        url (str): The URL of the webpage to fetch.

    Returns:
        str: The decoded UTF-8 content of the requested page.

    Raises:
        requests.RequestException: If an error occurs during the HTTP request.
    """
    reader_url_prefix = 'https://r.jina.ai/'
    request_url = reader_url_prefix + url
    response = requests.get(request_url)
    response.raise_for_status()  # optional but recommended for robustness
    return response.content.decode('utf-8')


In [6]:
get_page_content('https://datatalks.club')

"Title: Welcome to DataTalks.Club\n\nURL Source: https://datatalks.club/\n\nPublished Time: Tue, 21 Oct 2025 12:52:05 GMT\n\nMarkdown Content:\nWelcome to DataTalks.Club\n\n===============\n\nAI Dev Tools Zoomcamp: Learn AI-powered coding assistants and agents[Register here!](https://airtable.com/appJRFiWKHBgmEt70/shrpw7rk55Ewr1jCG)\n\nDataTalks.Club\n--------------\n\n[Articles](https://datatalks.club/articles.html)[Slack](https://datatalks.club/slack.html)[Events](https://datatalks.club/events.html)[Podcast](https://datatalks.club/podcast.html)[Books](https://datatalks.club/books.html)[Courses](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html)\n\n* * *\n\nThe place to talk about data\n============================\n\nGlobal online community of data science professionals, ML engineers, and AI practitioners\n-----------------------------------------------------------------------------------------\n\nSubscribe to our weekly newsletter and join our Slack.\n\

In [17]:
from agents import Agent, function_tool

In [18]:
assistant_instruction="""
You're a helpful assistant that helps answer user questions
"""

assistant = Agent(
    name='assistant',
    tools=[function_tool(fetch_url)],
    instructions = assistant_instruction,
    model='gpt-4o-mini'
)

In [19]:
from agents import Runner
runner = Runner()

In [20]:
user_prompt = "Summarize the content of https://openai.github.io/openai-agents-python/"
result = await runner.run(assistant, input=user_prompt)

In [22]:
print(result.new_items[-1].raw_item.content[0].text)

The **OpenAI Agents SDK** is a Python framework designed for creating AI applications with agents that can execute tasks effectively. Here are the key features and contents:

### Overview
- The SDK allows developers to build AI applications using simple components:
  - **Agents**: Language models (LLMs) combined with instructions and tools.
  - **Handoffs**: Mechanisms to delegate tasks to other agents.
  - **Guardrails**: Systems to validate inputs and outputs.
  - **Sessions**: Automatic management of conversation history.

### Design Principles
1. **Usability**: Aimed at being user-friendly while providing essential features.
2. **Customization**: Offers flexibility for users to define behaviors.

### Features
- **Agent Loop**: Handles interactions with tools and LLMs through an efficient looping mechanism.
- **Python Integration**: Utilizes Python’s features for easier orchestration.
- **Function Tools**: Allows any Python function to serve as a tool with schema validation.
- **Tra

In [23]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()
runner = OpenAIAgentsSDKRunner(
    chat_interface = chat_interface,
    agent = assistant
)

In [24]:
await runner.run();

You: summarize tagesschau.de


You: what is the main article today?


You: what are agents


You: stop


Chat ended.


###Youtube

In [25]:
from youtube_transcript_api import YouTubeTranscriptApi

def format_timestamp(seconds:float) -> str:
    """Convert seconds to H:MM:SS if > 1hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes,secs = divmod(remainder, 60)

    if hours>0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"

def make_subtitles (transcript) ->str:
    lines=[]

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n',' ')
        lines.append(ts+' '+text)
    return '\n'.join(lines)

def fetch_transcript_raw(video_id):
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id)
    return transcript

def fetch_transcript_text(video_id):
    transcript = fetch_transcript_raw(video_id)
    subtitles = make_subtitles(transcript)
    return subtitles

In [26]:
from pathlib import Path

def fetch_transcript_cached(video_id):
    cache_dir = Path("../_import/src/data_cache/youtube_videos")
    cache_file = cache_dir / f"{video_id}.txt"

    if cache_file.exists():
        return cache_file.read_text(encoding="utf-8")

    subtitles = fetch_transcript_text(video_id)
    cache_file.write_text(subtitles, encoding="utf-8")

    return subtitles

In [33]:
video_id='bwfR9dyxf1M'
cache_dir = Path("../_import/src/data_cache/youtube_videos")
cache_file = cache_dir / f"{video_id}.txt"

In [34]:
cache_file.read_text(encoding="utf-8")

FileNotFoundError: [Errno 2] No such file or directory: '../_import/src/data_cache/youtube_videos/bwfR9dyxf1M.txt'

In [35]:
!ls

GitHubDoc.ipynb  _import	   docs.py
__pycache__	 agenticrag.ipynb  webyoutube.ipynb


In [27]:
transcript = fetch_transcript_cached('vK_SxyqIfwk')
print(subtitles[:500])

RequestBlocked: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=vK_SxyqIfwk! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-ipblocked-exception).
2. (NOT RECOMMENDED) If you authenticate your requests using cookies, you will be able to continue doing requests for a while. However, YouTube will eventually permanently ban the account that you have used to aut

In [17]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return fetch_transcript_cached(video_id)


In [18]:
summary_instructions = """
You're a helpful assistant that helps answer user questions
about YouTube videos
"""

tools = [
    function_tool(fetch_youtube_transcript)
]

youtube_assistant = Agent(
    name='youtube_assistant',
    tools=tools,
    instructions=summary_instructions,
    model='gpt-4o-mini'
)

In [ ]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=youtube_assistant
)

await runner.run();

You: Summarize the YouTube video with ID 'vK_SxyqIfwk'
